## Below is the logic from the lambda function of the .yml file given in the blog for the stack formation in cloudformation

In [ ]:
import json
import boto3
import base64
from datetime import datetime
import random
import string
import os

s3_client = boto3.client('s3')
bedrock_client = boto3.client( service_name="bedrock-runtime", region_name="us-east-1" )

def handler(event, context):
  body = json.loads(event['body'])
  prompt = body['prompt']
  
  image_base64 = generate_image_at(prompt)
  
  image_eval = evaluate_image(prompt,image_base64)
  
  parsed_eval = json.loads(image_eval)
  
  # Decode the base64 image
  image_data = base64.b64decode(image_base64)
  
  bucket_name = os.environ['S3_BUCKET_NAME']
  
  # Generate PreSigned URL
  presigned_url = generate_presigned_url(bucket_name, image_data)
  
  return {
      'statusCode': 200,
      'body': json.dumps({
          'url': presigned_url,
          'evaluation_data': parsed_eval
          }),
      'headers': {
          'Content-Type': 'application/json',
      },
  }

# Bedrock call to Amazon Titan Image Generator G2
def generate_image_at(prompt):

  print("Using Amazon Titan Image Generator G2")

  body = json.dumps({
          "taskType": "TEXT_IMAGE",
          "textToImageParams": {
              "text": prompt,
              "negativeText": "bad quality, low res"
          },
          "imageGenerationConfig": {
              "numberOfImages": 1,
              "height": 1280,
              "width": 768,
              "cfgScale": generate_random_double(),
              "seed": generate_random_int()
          }
  })

  model_id = 'amazon.titan-image-generator-v2:0'
  accept = "application/json"
  content_type = "application/json"

  response = bedrock_client.invoke_model(
      body=body, modelId=model_id, accept=accept, contentType=content_type
  )
  response_body = json.loads(response.get("body").read())

  finish_reason = response_body.get("error")

  if finish_reason is not None:
      print(f"Image generation error. Error is {finish_reason}")

  generated_image = response_body.get("images")[0]

  return generated_image

def evaluate_image(prompt, image_base64):

  body = json.dumps(
      {
          "anthropic_version": "bedrock-2023-05-31",
          "max_tokens": 2048,
          "messages": [
              {
                  "role": "user",
                  "content": [
                      {
                          "type": "image",
                          "source": {
                              "type": "base64",
                              "media_type": "image/jpeg",
                              "data": image_base64
                          },
                      },
                      {"type": "text", 
                      "text": ( f"Evaluate the attached image against the original prompt: {prompt}. "
                                                  "Provide a short description of the generated image, give it score from 1 to 10, "
                                                  "share the reason of your scoring (maximum 15 words), and suggest how the picture could better attain to the original prompt (maximum 20 words). "
                                                  "Format your response like the following JSON structure: {{\"description\": \"x\", \"score\": \"y\", \"reason\": \"z\", \"suggestions\": \"w\"}} " )
                      },
                  ],
              }
          ],
      }
  )

  modelId = "anthropic.claude-3-5-sonnet-20240620-v1:0"
  accept = "application/json"
  contentType = "application/json"

  response = bedrock_client.invoke_model(
      body=body, modelId=modelId, accept=accept, contentType=contentType
  )
  
  response_body = json.loads(response.get("body").read())
  
  evaluation_text = response_body["content"][0]["text"]
  
  return evaluation_text

def generate_presigned_url(bucket_name, image_data):
  # Generate a unique object key using the current timestamp
  object_key = f"image-gen-eval/{generate_random_string(10)}.png"
  # Upload the image to S3
  s3_client.put_object(Bucket=bucket_name, Key=object_key, Body=image_data, ContentType='image/png')
  # Generate a pre-signed URL for the uploaded image
  presigned_url = s3_client.generate_presigned_url('get_object', Params={'Bucket': bucket_name, 'Key': object_key}, ExpiresIn=3600)
  return presigned_url
  
def generate_random_string(length):
  # Define the characters to include in the random string
  chars = string.ascii_lowercase + string.digits
  # Generate the random string
  random_string = ''.join(random.choice(chars) for _ in range(length))
  return random_string

def generate_random_double(min_value=5.0, max_value=9.0):
  return random.uniform(min_value, max_value)

def generate_random_int(min_value=1, max_value=50):
  return random.randint(min_value, max_value)
